<a href="https://colab.research.google.com/github/yeonsub/models_from_scratch/blob/main/RNN_Edu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn

input_size = 5
batch_size = 1
sequence_length = 4
hidden_size = 32

rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size)

print(f'rnn.weight_hh_l0: {rnn.weight_hh_l0.shape}')
print(f'rnn.weight_ih_l0: {rnn.weight_ih_l0.shape}')
print(f'rnn.bias_ih_l0: {rnn.bias_ih_l0.shape}')
print(f'rnn.bias_ih_l0: {rnn.bias_ih_l0.shape}')

input_tensor = torch.ones(sequence_length, batch_size, input_size)
hidden_tensor = torch.zeros(1, batch_size, hidden_size)

#print(input_tensor)
output, hidden = rnn(input_tensor, hidden_tensor)

#print(output)
#print(hidden)

rnn.weight_hh_l0: torch.Size([32, 32])
rnn.weight_ih_l0: torch.Size([32, 5])
rnn.bias_ih_l0: torch.Size([32])
rnn.bias_ih_l0: torch.Size([32])


In [ ]:
class EmotionRNN(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(EmotionRNN, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size)
        self.dense = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_tensor, hidden_tensor):
        rnn_output, _ = self.rnn(input_tensor, hidden_tensor)
        last_one = rnn_output[-1]
        output = self.dense(last_one)
        logit = self.softmax(output)
        return logit




In [17]:
import numpy as np

dataset = [
    ["I Love You", "Good"],
    ["I Hate You", "Bad"],
]

word_to_index_map = {
    "I": 0,
    "Love": 1,
    "Hate": 2,
    "You": 3,
}

label_to_index_map = {
    "Good": 0,
    "Bad": 1
}

n_epoch = 10
output_size = 2

model = EmotionRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epoch):
    totalloss=0
    for data in dataset:
        sentence = data[0]
        target = data[1]
        #get word index from dictionary table
        sentence_to_index = [word_to_index_map[word] for word in sentence.split()]

        sentence_to_one_hot_vector = [np.eye(4)[index] for index in sentence_to_index]
        # print(sentence_to_one_hot_vector)

        input_tensor = torch.tensor(sentence_to_one_hot_vector, dtype=torch.float)
        #print(input_tensor)

        input_tensor = input_tensor.reshape(3, 1, 4)
        #print(input_tensor)
        target_to_index = label_to_index_map[target]
        target_to_one_hot_vector = np.eye(2)[target_to_index]
        target_tensor = torch.tensor(target_to_one_hot_vector, dtype=torch.float)
        target_tensor = target_tensor.reshape(1, -1)

        hidden_state = torch.zeros(1, 1, hidden_size, dtype=torch.float)
        prediction = model(input_tensor, hidden_state)

        loss = criterion(prediction, target_tensor)
        loss.backward()
        optimizer.step()
        totalloss +=loss.item()

    print(totalloss)

1.3883991837501526
1.3806430101394653
1.3730066418647766
1.3644734621047974
1.354790210723877
1.3438947796821594
1.3318548202514648
1.3187695741653442
1.304731547832489
1.2898257374763489


In [ ]:
#test
